ich konnte es noch nicht laufen lassen weil zu viele observations drin sind, vllt kannst du es mal probieren ? :)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
!pip install minisom
from minisom import MiniSom
import math 

In [0]:
# load data
data =pd.read_csv('/content/gdrive/My Drive/data_wordvectors_obs.csv', sep=',').iloc[:, 1:]
data

In [0]:
indexvec = []
for col in range(0, len(data.columns)-7): 
  indexvec.append("x" + str(col+1))
indexvec
vectors = data[indexvec]
vectors = np.array(vectors)

In [0]:
category_color = {'CDU' : 'black',
                 'SPD': 'red',
                 'Grüne' : 'green',
                 'AfD' : 'brown',
                 'Linke': 'darkred',
                 'FDP' : 'yellow'}
  
category_color2 = {'Annegreth Kramp-Karrenbauer': 'black',
                  'Angela Merkel': 'black',
                  'Katharina Barley': 'red',
                  'Andrea Nahles': 'red',
                   "Cem Özdemir":'limegreen',
                 "Kathrin Goering-Eckardt":'limegreen',
                 "Alice Weidel": 'brown',
                 "Jörg Meuthen": 'brown',
                 "Sarah Wagenknecht": 'darkred',
                 "Christian Lindner": 'yellow'}
category_short = {'Annegreth Kramp-Karrenbauer': 'AKK',
                  'Angela Merkel': 'AM',
                  'Katharina Barley': 'KB',
                  'Andrea Nahles': 'AN',
                   "Cem Özdemir":'CÖ',
                 "Kathrin Goering-Eckardt":'KGE',
                 "Alice Weidel": 'AW',
                 "Jörg Meuthen": 'JM',
                 "Sarah Wagenknecht": 'SW',
                 "Christian Lindner": 'CL'}

In [20]:
# choose map dimension as number of samples so that
# each neuron gets mapped to a sample
# in our case: one tweet
map_dim = round(5*math.sqrt(len(vectors)))
#map_dim = 5
print("Preparing training...")
som = MiniSom(map_dim, map_dim, 30, sigma=1.0, random_seed=1)
som.pca_weights_init(vectors)
print("\n...ready!")

Training...

...ready!


In [21]:
# train SOM
# due to our large observation size this might take some time...
som.train_batch(vectors, len(vectors)*10)

KeyboardInterrupt: ignored

In [0]:
user_map = som.labels_map(vectors, data['real'])

In [0]:
user_map.items()

In [0]:
plt.figure(figsize=(map_dim, map_dim))
plt.pcolor(som.distance_map().T, cmap='gray_r', alpha=.2)
plt.xticks(np.arange(map_dim+1))
plt.yticks(np.arange(map_dim+1))
plt.grid()

for p, countries in user_map.items():
    countries = list(countries)
    x = p[0] + .5
    y = p[1] - .5
    for i, c in enumerate(countries):
        off_set = (i+1)/len(countries) - 0.05
        plt.text(x, y+off_set, category_short[c], color = category_color2[c],  fontsize=10)
        #plt.plot(x, y+off_set, 'ro', color = category_color2[c],)
legend_elements = [Patch(facecolor=clr,
                         edgecolor='w',
                         label=l) for l, clr in category_color.items()]
plt.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, .95))
plt.show()